In [261]:
# reload during dev
from importlib import reload
import PET_raw_data
PET_raw_data = reload(PET_raw_data)


import PET_raw_data
dict_load = {'.\data\[Adt Income] adult.csv'      : {'describe_params' : {'missing_level' : {'data': {'?'}}}}
            ,'.\data\[Census] adult.data'         : {'read_params'     : {'header_exist' : 'N'
                                                                         ,'header'       : ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
                                                                         }
                                                    ,'describe_params' : {'missing_level' : {'data': {'?'}}}
                                                    }
            ,'.\data\[NHANES] B.csv'              : {'read_params'     : {'header_exist' : 'N'
                                                                         ,'header'       : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                                                                         }
                                                    }
            ,'.\data\[Retail] Online Retail.xlsx' : {'read_method'     : 'pandas_xlsx'
                                                    ,'read_params'     : {'sheet_name' : 'Online Retail'
                                                                         ,'str_col'    : ['InvoiceNo','StockCode','InvoiceDate','CustomerID']
                                                                         }
                                                    }
            ,'.\data\[UK-US PF] va_household_ver_1_9_instance_2.csv' : {}
            ,'.\data\[UK-US PF] va_person_ver_1_9_instance_2.csv'    : {}
            }

for filepath ,file_params in dict_load.items():
    file_params_init = {'describe'        : 'Y'
                       ,'describe_params' : {'correlation'   : 'Y'
                                            ,'collinearity'  : 'Y'}
                       ,'report'          : 'Y'
                       ,'report_params'   : {'print_report' : 'N'
                                            ,'save_report'  : 'Y'}
                       }
    file_params_init.update(file_params)
    print(f"Now is {filepath}")
    __temp = PET_raw_data.load_data(filepath ,file_params_init)

Now is .\data\[Adt Income] adult.csv
Now is .\data\[Census] adult.data
Now is .\data\[NHANES] B.csv
Now is .\data\[Retail] Online Retail.xlsx


In [260]:
pd.read_excel('.\data\[Retail] Online Retail.xlsx').head(6)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
